# AutoGen Assistant Agent

## Install Library
Get Latest Version here: https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/installation.html

In [1]:
%pip install "autogen-agentchat==0.4.2" "autogen-ext[openai,azure,docker]==0.4.2"

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Load Azure Configuration

In [9]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client
Using the model client class

In [10]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)

## Defining an Agent
AssistantAgent is a built-in agent that uses a language model and has the ability to use tools.

In [11]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken

# Define a tool
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 73 degrees and Sunny."

# Define an agent
weather_agent = AssistantAgent(
    name="weather_agent",
    description="This agent provides weather information.",
    model_client=az_model_client,
    tools=[get_weather],
    system_message="""
    You are agent that assists with weather information.
    If you are given an image, you will identify the city in the image and provide the weather information. 
    If you are asked about other topics, tell the user you can only provide weather information.
    """,
)

## Text Message
Create a text message, which accepts a string content and a string source

In [ ]:
from autogen_agentchat.messages import TextMessage

text_message = TextMessage(content="What's the weather in Chicago", source="User")

# Run the team and stream messages to the console.
stream = weather_agent.run_stream(task=text_message)
await Console(stream)

## MultiModal Message

In [ ]:
from autogen_agentchat.messages import MultiModalMessage
from autogen_core import Image as AGImage
from PIL import Image
from matplotlib import pyplot as plt
import cv2

img_path = "../Data/images/AdobeStock_186373174.jpeg"

# Load the image using OpenCV.
img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
# Convert the image from BGR to RGB for displaying with matplotlib, because OpenCV uses BGR by default and matplotlib expects RGB.
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Display the image with matplotlib.
plt.imshow(img_rgb)
plt.axis("off")  # Turn off axis labels.
plt.show()

raw_image = Image.open(img_path)
# Create an AGImage object from the raw image.
img = AGImage(raw_image)
multi_modal_message = MultiModalMessage(content=["What's the weather in this image?", img], source="User")

# Run the team and stream messages to the console.
stream = weather_agent.run_stream(task=multi_modal_message)
await Console(stream)

## Agents have State

In [ ]:
text_message = TextMessage(content="What were the cities again that I asked about the weather?", source="User")

# Run the team and stream messages to the console.
stream = weather_agent.run_stream(task=text_message)
await Console(stream)

## Getting Responses

1. on_messages() method returns a Response that contains the following attributes:
- chat_message: contains agent’s final response 
- inner_messages: stores the agent’s “thought process” that led to the final response.
2. on_messages() will update the internal state of the agent – it will add the messages to the agent’s history. 
- You should call this method with new messages.
- You should NOT repeatedly call this method with the same messages or the complete history.

In [ ]:
img_path = "../Data/images/AdobeStock_192483501.jpeg"

img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
# Convert the image from BGR to RGB for displaying with matplotlib, because OpenCV uses BGR by default and matplotlib expects RGB.
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Display the image with matplotlib.
plt.imshow(img_rgb)
plt.axis("off")  # Turn off axis labels.
plt.show()

raw_image = Image.open(img_path )
# Create an AGImage object from the raw image.
img = AGImage(raw_image)
multi_modal_message = MultiModalMessage(content=["What's the city where this image is located? Provide the weather there.", img], source="User")

cancellation_token=CancellationToken()
response = await weather_agent.on_messages(
    [multi_modal_message],
    cancellation_token=cancellation_token,
)
# contains the agent's "thought process"
print("Thought process: ")
print(response.inner_messages)
print("\n")
# contains the agent's final response
print("Final Response: ")
print(response.chat_message)
print("\n")
print("Final Response Content: ")
print(response.chat_message.content)

## Streaming Messages
We can also stream each message as it is generated by the agent by using the on_messages_stream() method, and use Console to print the messages as they appear to the console.

In [ ]:
text_message = TextMessage(content="What were the cities again that I asked about the weather?", source="User")

from IPython.display import display

cancellation_token=CancellationToken()
await Console(weather_agent.on_messages_stream(
        [text_message],
        cancellation_token=cancellation_token,)
    )

## Reset Agent to Initial State

In [ ]:
cancellation_token=CancellationToken()
# Reset the agent
await weather_agent.on_reset(cancellation_token=cancellation_token)

text_message = TextMessage(content="What were the cities again that I asked about the weather?", source="User")
response = await weather_agent.on_messages(
        [text_message],
        cancellation_token=cancellation_token,
    )
print(response.chat_message.content)

# Reset the agent again
await weather_agent.on_reset(cancellation_token=cancellation_token)



## Using Model Context
AssistantAgent has a model_context parameter that can be used to pass in a ChatCompletionContext object. This allows the agent to use different model contexts, such as BufferedChatCompletionContext to limit the context sent to the model.

By default, AssistantAgent uses the UnboundedChatCompletionContext which sends the full conversation history to the model. To limit the context to the last n messages, you can use the BufferedChatCompletionContext.

In [18]:
from autogen_core.model_context import BufferedChatCompletionContext
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage

# Define a tool
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 73 degrees and Sunny."

# Define an agent
weather_agent = AssistantAgent(
    name="weather_agent",
    description="This agent provides weather information.",
    model_client=az_model_client,
    tools=[get_weather],
    system_message="""
    You are agent that assists with weather information.
    """,
    model_context=BufferedChatCompletionContext(buffer_size=8),  
)

In [ ]:
text_message = TextMessage(content="What is the weather in Chicago?", source="User")
stream = weather_agent.run_stream(task=text_message)
await Console(stream)
print("\n")

text_message = TextMessage(content="How about in New York?", source="User")
stream = weather_agent.run_stream(task=text_message)
await Console(stream)
print("\n")

text_message = TextMessage(content="What's it like in Tokyo?", source="User")
stream = weather_agent.run_stream(task=text_message)
await Console(stream)
print("\n")

In [ ]:
text_message = TextMessage(content="What was the weather in the first city again?", source="User")
stream = weather_agent.run_stream(task=text_message)
await Console(stream)
print("\n")

## New York is the first city in the context given that each interaction was 4 messages.

# Save and Load Agent State

In [ ]:
import json

state = await weather_agent.save_state()

# (Optional) Write state to disk.
with open("assistant_state.json", "w") as f:
    json.dump(state, f)

# (Optional) Load it back from disk.
with open("assistant_state.json", "r") as f:
    state = json.load(f)
    print(state) # Inspect the state, which contains the chat history.

# Carry on the chat.
response = await weather_agent.on_messages([TextMessage(content="What is the weather in Rome?", source="user")], cancellation_token=CancellationToken())
print(response.chat_message.content)


In [ ]:
# Carry on the same chat again.
response = await weather_agent.on_messages([TextMessage(content="What was the weather in the first city again?", source="user")], cancellation_token=CancellationToken())
print(response.chat_message.content)

In [ ]:

# Load the state, resulting the agent to revert to the previous state before the last message.
await weather_agent.load_state(state)

# Carry on the same chat again.
response = await weather_agent.on_messages([TextMessage(content="What was the weather in the first city again?", source="user")], cancellation_token=CancellationToken())
print(response.chat_message.content)